# MNIST CNN with Estimator

上一篇主要是 invoke high levle function 去建構模型

而這邊引入 estimator 的概念把 training 過程也換成 high level function，根據 TRAIN/ EVAL/ PREDICT 三種狀態會有不同的 output

- Estimator
- loggin hook

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist_data = input_data.read_data_sets('data/mnist', one_hot=True)

import os
import argparse
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from pathlib import Path
print(f"tensorflow version {tf.__version__}")

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/mnist/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/mnist/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting data/mnist/t10k-images-idx3-ubyte.gz
Extracting data/mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
tensorflow version 1.11.0


In [2]:
def argparser():
    parser = argparse.ArgumentParser()
    
    # training parameter
    parser.add_argument('--gpu', dest='gpu', required=True, \
                        help='require one GPU to execute')
    parser.add_argument('--lr', dest='lr', \
                        default=0.1, type=float, \
                        help='learning rate')
    parser.add_argument('--steps', dest='steps', \
                        default=500, type=int, \
                        help='number of update steps')
    parser.add_argument('--epochs', dest='epochs', \
                        type=int, \
                        help='number of epochs')
    parser.add_argument('--bz', dest='bz', \
                        default=128, type=int, \
                        help='batch size')
    parser.add_argument('--display-steps', dest='display_steps', \
                        default=100, type=int, \
                        help='display steps')
    
    # network parameter
    parser.add_argument('--filters', dest='filters', \
                        default=[256, 256], nargs='+', type=int, \
                        help='hidden neurons')
    parser.add_argument('--input-shape', dest='input_shape', \
                        default=784, nargs='+', type=int)
    parser.add_argument('--num-class', dest='n_output', \
                        default=10, type=int)
    parser.add_argument('--ckpt-savedir', dest='ckpt_savedir', \
                        default='outputs/02_mnist_estimator')
    return parser

# setting args
args = argparser().parse_args([
    '--gpu', '3',
    '--lr', '0.0001',
    '--input-shape', '784',
    '--filters', '512', '256', '256'
])
print(args)
os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu

Namespace(bz=128, ckpt_savedir='outputs/02_mnist_estimator', display_steps=100, epochs=None, filters=[512, 256, 256], gpu='3', input_shape=[784], lr=0.0001, n_output=10, steps=500)


In [3]:
def cnn_model_fn(features, labels, mode):
    """build model graph"""
    # input layer
    input_layer = tf.reshape(features, [-1, 28, 28, 1], name='input')
    
    # conv1
    conv1 = tf.layers.conv2d(input_layer, filters=32, kernel_size=5, padding='same', name='conv_1')
    conv1 = tf.layers.batch_normalization(conv1, name='bn_1')
    conv1 = tf.nn.relu(conv1, name='relu_1')
    pool1 = tf.layers.max_pooling2d(conv1, pool_size=2, strides=2)
    
    # conv2
    conv2 = tf.layers.conv2d(pool1, filters=64, kernel_size=5, padding='same', name='conv_2')
    conv2 = tf.layers.batch_normalization(conv2, name='bn_2')
    conv2 = tf.nn.relu(conv2, name='relu_2')
    
    # dense layer
    flat = tf.layers.flatten(conv2, name='flatten')
    dense = tf.layers.dense(flat, 1024, activation=tf.nn.relu, name='dense')
    dropout = tf.layers.dropout(inputs=dense, rate=0.5, training= (mode == tf.estimator.ModeKeys.TRAIN))
    
    # output layer (logits)
    y_pred = tf.layers.dense(dropout, 10, name='output')
    
    # outputs
    predictions = {
        # generate predictions (for PREDICT and EVAL mode)
        'classes': tf.argmax(y_pred, axis=1),
        # setting name to the graph is used for PREDICT by `logging_hook`
        'probabilities': tf.nn.softmax(y_pred, name='softmax_tensor')
    }
    
    # calc loss (for both TRAIN and EVAL)
    print(f"y_pred.shape {y_pred.shape}, labels.shape {labels.shape}")
    loss = tf.losses.sparse_softmax_cross_entropy(
        logits=y_pred, labels=tf.argmax(labels, axis=1))

    """config diff output for diff mode"""
    # TRAIN mode: training operation
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=1e-4)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # EVAL mode: evaluation metrics
    elif mode == tf.estimator.ModeKeys.EVAL:
        eval_metric_ops = {
            'accuracy': tf.metrics.accuracy(
                labels=tf.argmax(labels, axis=1), predictions=predictions['classes'])
        }
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)
    
    # PREDICT mode: inference result
    elif mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    else:
        raise ValueError(f"There's no mode for {mode}")

In [4]:
"""main"""
# load data
train_data = mnist_data.train.images
train_labels = np.asarray(mnist_data.train.labels, dtype=np.int32)
eval_data = mnist_data.test.images
eval_labels = np.asarray(mnist_data.test.labels, dtype=np.int32)

# create estimator
mnist_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir=args.ckpt_savedir)

# setup logging for predictions
tensor_to_log = {'probabilities': 'softmax_tensor'}
logging_hook = tf.train.LoggingTensorHook(
    tensors=tensor_to_log, every_n_iter=50)

# train model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x=train_data,
    y=train_labels,
    batch_size=args.bz,
    num_epochs=args.epochs,
    shuffle=True
)
mnist_classifier.train(
    input_fn=train_input_fn,
    steps=args.steps,
    hooks=[logging_hook]
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'outputs/02_mnist_estimator', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f2b7a45feb8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pi

In [5]:
# evaluate the model and print the result
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x=eval_data,
    y=eval_labels,
    num_epochs=1,
    shuffle=False
)
eval_result = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_result)

INFO:tensorflow:Calling model_fn.
y_pred.shape (?, 10), labels.shape (?, 10)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-01-25-03:40:40
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from outputs/02_mnist_estimator/model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-01-25-03:40:41
INFO:tensorflow:Saving dict for global step 2000: accuracy = 0.9887, global_step = 2000, loss = 0.032307748
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 2000: outputs/02_mnist_estimator/model.ckpt-2000
{'accuracy': 0.9887, 'loss': 0.032307748, 'global_step': 2000}
